# ADM Homework 3

## Search Engine

In [3]:
if True:
    %pip install -qqq numpy scipy matplotlib pandas scikit-learn nltk forex_python babel

Note: you may need to restart the kernel to use updated packages.


In [5]:
import string
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import re
from forex_python.converter import CurrencyRates
from babel.numbers import get_currency_symbol
from nltk.stem import *
from collections import Counter
from functools import reduce
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import heapq

In [23]:
nltk.download('stopwords')
nltk.download('punkt')
stemmer = PorterStemmer()
masters = pd.read_json(r"C:\Users\Elias Antoun\Downloads\dataset.json")

[nltk_data] Downloading package stopwords to C:\Users\Elias
[nltk_data]     Antoun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Elias
[nltk_data]     Antoun\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Preprocessing 2.0

In [24]:
# 2.0.0 Tokenize, remove stopwords and punctuation, and stem

stop_words = set(stopwords.words('english'))

def tok_punct_and_stop(text):
    # tokenize the text given
    desc_tok = word_tokenize(text)

    # remove punctuation and stopwords from the tokens
    desc_tok_nostop_nopunct = [x for x in desc_tok if x.lower() not in stop_words and x not in string.punctuation]

    # stem using porterstemmer
    stemmed_desc = [stemmer.stem(x) for x in desc_tok_nostop_nopunct]
    return stemmed_desc

# Apply the function to the description column of our dataset

masters["description"] = masters["description"].apply(tok_punct_and_stop)

In [25]:
masters

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,"[3d, visualis, anim, play, role, mani, area, p...",September,Please see the university website for further ...,MSc,1 year full-time,Glasgow,United Kingdom,On Campus,
1,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,"[busi, govern, reli, sound, financi, knowledg,...",September,"UK: £18,000 (Total) International: £34,750 (To...",MSc,1 year full time,Leeds,United Kingdom,On Campus,
2,Accounting and Finance (MSc),University of Bath,School of Management,Full time,"[develop, in-depth, knowledg, account, financ,...",September,\n</div,MSc,1 year full-time,Bath,United Kingdom,On Campus,
3,"Accounting, Accountability & Financial Manag...",King’s College London,King’s Business School,Full time,"[account, account, financi, manag, msc, cours,...",September,Please see the university website for further ...,MSc,1 year FT,London,United Kingdom,On Campus,
4,"Accounting, Financial Management and Digital...",University of Reading,Henley Business School,Full time,"[embark, profession, account, career, academ, ...",September,\n</div,MSc,1 year full time,Reading,United Kingdom,On Campus,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1825,Cancer Biology (MSc),Sheffield Hallam University,Postgraduate Courses,Full time,"[work, state-of-the-art, facil, gain, in-depth...",September,Please see the university website for further ...,MSc,"1 year full-time, 2 years part-time",Sheffield,United Kingdom,On Campus,
1826,Cancer Biology and Immunology (MSc),University of Southampton,Faculty of Medicine,Full time,"[embark, career, cancer, research, focu, lates...",See Course,Please see the university website for further ...,MSc,1 year full-time; 2 years part-time,Southampton,United Kingdom,On Campus,
1827,Cancer Biology and Precision Oncology (Onlin...,University of Edinburgh,Edinburgh Medical School,Part time,"[programm, descriptionthi, new, programm, equi...",September,"For 2022/23 the fees are £6,035.",MSc,1-3 years part-time,Edinburgh,United Kingdom,Online,
1828,Cancer Biology and Therapeutics - MSc,Cardiff University,Cardiff School of Pharmacy and Pharmaceutical ...,Full time,"[studi, coursegradu, cours, breadth, depth, ca...",September,Please see the university website for further ...,MSc,1 year full time,Cardiff,United Kingdom,On Campus,


In [26]:
# 2.0.1 fee preprocessing

# Step 1: Currency management setup


# Dictionary to standardize possible ways of writing a currency

curr_codes = {"EUR": ["euros", "€", "eur", "euro"],
              "USD": ["usd", "dollars", "$", "dollar"],
              "GBP": ["gbp", "pounds", "pound", "£"]}


# Step 2: amount and currency extraction
def extract_amount_and_curr(text):
    currencies = ["usd", "$", "dollars", "dollar", "euro", "euros", "€", "gbp", "eur", "£", "pounds"]
    amount = []
    i = 0
    text_tok = tok_punct_and_stop(text)
    tex_l = []
    for entry in text_tok:
        # Find the index of the first digit
        digit_index = next((i for i, c in enumerate(entry) if c.isdigit()), None)

        if digit_index is not None:
            # Split the entry into two parts
            prefix = entry[:digit_index]
            suffix = entry[digit_index:]

            # Add the parts to the result list
            tex_l.append(prefix)
            tex_l.append(suffix)
        else:
            # If no digit is found, keep the original entry
            tex_l.append(entry)

    # parse through obtained list to find numbers and currency symbols
    while i < len(tex_l):
        if tex_l[i].lower() in currencies:
            if i - 1 >= 0 and tex_l[i - 1].replace(',', '').isnumeric():
                amount.append(tex_l[i - 1].replace(',', ''))
            elif i + 1 < len(tex_l) and tex_l[i + 1].replace(',', '').isnumeric():
                amount.append(tex_l[i + 1].replace(',', ''))
            amount.append(tex_l[i])
        i += 1
    return amount


masters["fees"] = masters["fees"].apply(extract_amount_and_curr)


# Step 3: define function to return the max amount, and its currency

def max_standard(amount):
    curr = []
    price = [float(x) for x in amount if x.isnumeric()]
    for y in amount:
        for key, values in curr_codes.items():
            if y.lower() in values:
                curr.append(key)

    return sorted(price, reverse=True), curr


masters["fees"] = masters["fees"].apply(max_standard)

# Step 4: Conversion


def convert_to_euros(amount, source_currency):
    c = CurrencyRates()
    try:
        exchange_rate = c.get_rate(source_currency, 'EUR')
        return amount * exchange_rate
    except Exception as e:
        print(f"Error: {e}")
        return None

def max_amounts_in_euros(fees_column):
    max_amounts = []

    for entry in fees_column:
        amounts, currencies = entry
        max_amount_euros = float('-inf')

        for amount, currency in zip(amounts, currencies):
            amount_euros = convert_to_euros(amount, currency)
            if amount_euros is not None:
                max_amount_euros = max(max_amount_euros, amount_euros)

        max_amounts.append(max_amount_euros)

    return max_amounts

masters['fees'] = max_amounts_in_euros(masters['fees'])
masters['fees'] = masters['fees'].apply(lambda x: f"{x:.2f}" if pd.notna(x) else np.nan)
print(masters["fees"])

0           -inf
1       39871.49
2           -inf
3           -inf
4           -inf
          ...   
1825        -inf
1826        -inf
1827     6924.44
1828        -inf
1829    17325.45
Name: fees, Length: 1830, dtype: object


### Build a vocabulary

In [27]:
vocabulary_alt = Counter(reduce(lambda x, y: x + y, masters.description.values)).keys()
index = {}
unique_id = 1
for word in list(vocabulary_alt):
  index[unique_id] = word
  unique_id+=1
index

{1: '3d',
 2: 'visualis',
 3: 'anim',
 4: 'play',
 5: 'role',
 6: 'mani',
 7: 'area',
 8: 'popular',
 9: 'media',
 10: 'keep',
 11: 'grow',
 12: 'digit',
 13: 'provid',
 14: 'eye-catch',
 15: 'special',
 16: 'effect',
 17: '21st',
 18: 'centuri',
 19: "'s",
 20: 'favourit',
 21: 'film',
 22: 'televis',
 23: 'show',
 24: 'design',
 25: 'also',
 26: 'essenti',
 27: 'everyday',
 28: 'work',
 29: 'everyth',
 30: 'comput',
 31: 'game',
 32: 'develop',
 33: 'onlin',
 34: 'virtual',
 35: 'world',
 36: 'industri',
 37: 'market',
 38: 'product',
 39: 'architecture.gcu',
 40: 'programm',
 41: 'environ',
 42: 'help',
 43: 'skill',
 44: 'thrive',
 45: 'success',
 46: 'career',
 47: 'visual',
 48: 'practic',
 49: 'career-focus',
 50: 'orient',
 51: 'toward',
 52: 'current',
 53: 'need',
 54: 'technolog',
 55: 'prior',
 56: 'knowledg',
 57: 'requir',
 58: 'busi',
 59: 'govern',
 60: 'reli',
 61: 'sound',
 62: 'financi',
 63: 'underpin',
 64: 'strategi',
 65: 'success.thi',
 66: 'cours',
 67: 'kick-s

In [28]:
terms = pd.DataFrame(data=list(vocabulary_alt), columns=['term'])
terms['reverse'] = terms.term.apply(lambda item: list(masters.loc[masters.description.apply(lambda row: item in row)].index)) #apply instead of progress_apply
terms.head()

,term,reverse
0,3d,"[0, 505, 586, 668, 669, 962]"
1,visualis,"[0, 92, 94, 458, 813, 1147, 1351, 1367, 1495, ..."
2,anim,"[0, 14, 28, 37, 144, 202, 310, 690, 754, 1041,..."
3,play,"[0, 24, 45, 94, 107, 232, 249, 338, 367, 380, ..."
4,role,"[0, 17, 18, 19, 24, 48, 81, 83, 94, 96, 101, 1..."


In [29]:
def find_word_in_documents(word):
    filtered_rows = terms[terms['term'] == word]['reverse'].values[0]
    result_df = masters.loc[filtered_rows, ['courseName', 'universityName', 'description', 'url']]
    return result_df

word_to_search = "account"
f_n_words = find_word_in_documents(word_to_search)
f_n_words

,courseName,universityName,description,url
1,Accounting and Finance - MSc,University of Leeds,"[busi, govern, reli, sound, financi, knowledg,...",
2,Accounting and Finance (MSc),University of Bath,"[develop, in-depth, knowledg, account, financ,...",
3,"Accounting, Accountability & Financial Manag...",King’s College London,"[account, account, financi, manag, msc, cours,...",
4,"Accounting, Financial Management and Digital...",University of Reading,"[embark, profession, account, career, academ, ...",
55,Cardiovascular Sciences - MSc(MedSci),University of Glasgow,"[cardiovascular, diseas, project, remain, sing...",
...,...,...,...,...
1699,Building Services Engineering Management MSc,Brunel University London,"[ask, brunelbrunel, 's, build, servic, engin, ...",
1714,Business Administration MSc,University of Groningen,"[studi, busi, administr, groningen, choos, one...",
1720,Business Analysis & Finance - MSc,University of Leicester,"[busi, analysi, financ, msc, univers, leicest,...",
1727,Business Analytics - MSc,Xi’an Jiaotong-Liverpool University,"[rapid, growth, avail, data, transform, way, m...",


In [30]:
tfidf = TfidfVectorizer(input='content', lowercase=False, tokenizer=lambda text: text) # , max_df=0.1
results = tfidf.fit_transform(masters.description)
result_dense = results.todense()
tfidf_data = pd.DataFrame(result_dense.tolist(), index=masters.index, columns=tfidf.get_feature_names_out())
len(tfidf_data)
tfidf_data

c:\Users\Elias Antoun\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,'','big,'credit,'laboratori,'life,'ll,'master,'omic,'re,'s,...,﻿offer,﻿over,﻿overviewbuild,﻿overviewth,﻿the,﻿theoret,﻿thi,﻿want,﻿you,﻿your
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.129056,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.105174,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.296041,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.306805,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1825,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1826,0.0,0.0,0.0,0.0,0.0,0.316869,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1827,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1828,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
cossim_data =pd.DataFrame(cosine_similarity(tfidf_data), index=masters.index, columns=masters.index)
cossim_data

,0,1,2,3,4,5,6,7,8,9,...,1820,1821,1822,1823,1824,1825,1826,1827,1828,1829
0,1.000000,0.034014,0.060136,0.044689,0.043579,0.014578,0.019519,0.029198,0.036305,0.032033,...,0.073866,0.005557,0.005557,0.012136,0.009614,0.054766,0.021275,0.061069,0.030584,0.021406
1,0.034014,1.000000,0.341223,0.278438,0.323464,0.016036,0.059216,0.030356,0.056752,0.041865,...,0.043955,0.036420,0.036420,0.059596,0.057856,0.047040,0.053124,0.041559,0.049855,0.033845
2,0.060136,0.341223,1.000000,0.387080,0.188841,0.007032,0.082088,0.033526,0.103012,0.069877,...,0.049097,0.021392,0.021392,0.015267,0.053376,0.083374,0.140217,0.044993,0.031976,0.040390
3,0.044689,0.278438,0.387080,1.000000,0.296849,0.071447,0.063428,0.026073,0.046220,0.027944,...,0.112161,0.044607,0.044607,0.020753,0.027178,0.010953,0.122780,0.047366,0.027249,0.015700
4,0.043579,0.323464,0.188841,0.296849,1.000000,0.061442,0.048807,0.016635,0.047721,0.061011,...,0.143536,0.049655,0.049655,0.092666,0.056873,0.024414,0.046482,0.048136,0.016097,0.032295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1825,0.054766,0.047040,0.083374,0.010953,0.024414,0.000000,0.036750,0.028512,0.034399,0.041222,...,0.020280,0.147191,0.147191,0.235712,0.184009,1.000000,0.304568,0.154884,0.133600,0.156255
1826,0.021275,0.053124,0.140217,0.122780,0.046482,0.016192,0.017896,0.022244,0.058481,0.080424,...,0.016275,0.166807,0.166807,0.419493,0.209516,0.304568,1.000000,0.259300,0.145420,0.251419
1827,0.061069,0.041559,0.044993,0.047366,0.048136,0.011076,0.044856,0.024691,0.056632,0.036409,...,0.034865,0.136704,0.136704,0.197691,0.129151,0.154884,0.259300,1.000000,0.096718,0.189645
1828,0.030584,0.049855,0.031976,0.027249,0.016097,0.007698,0.052025,0.051381,0.071776,0.069785,...,0.022289,0.076015,0.076015,0.147124,0.088416,0.133600,0.145420,0.096718,1.000000,0.085441


In [32]:
def top_5_cosine_similarity_documents(word):
    word_documents = find_word_in_documents(word)
    document_indices = word_documents.index.tolist()
    cosine_scores = cossim_data.loc[document_indices].sum(axis=1)
    top_documents_indices = cosine_scores.nlargest(5).index
    top_documents_info = masters.loc[top_documents_indices, ['courseName', 'universityName', 'description', 'url']]
    return top_documents_info

word_to_search = "account"
top_document = top_5_cosine_similarity_documents(word_to_search)
top_document

,courseName,universityName,description,url
644,"Accounting, Finance and Management - MSc",University of Bristol,"[highli, success, msc, programm, design, stude...",
1699,Building Services Engineering Management MSc,Brunel University London,"[ask, brunelbrunel, 's, build, servic, engin, ...",
623,Accounting and Finance MSc,University of Greenwich,"[flexibl, master, ’, account, financ, give, st...",
612,Accounting and Finance - MSc,University of Sussex,"[msc, ’, advanc, account, financ, knowledg, pr...",
757,Advanced Clinical Practice (Postgraduate Cer...,University of Gloucestershire,"[programm, enabl, experienc, healthcar, profes...",


In [33]:
def create_inverted_index(tfidf_data):
    inverted_index = {}
    for term_id, term in enumerate(tfidf_data.columns):
        term_info = []
        for doc_id in tfidf_data.index:
            tfidf_score = tfidf_data.at[doc_id, term]
            if tfidf_score > 0:
                term_info.append((doc_id, tfidf_score))
        inverted_index[term_id + 1] = term_info
    return inverted_index

In [35]:
def find_related_documents(word, masters, tfidf_data, cossim_data, inverted_index):
    term_id = terms[terms['term'] == word].index[0] + 1
    term_info = inverted_index.get(term_id, [])
    if not term_info:
        print(f"No documents found for the word '{word}'.")
        return
    term_info.sort(key=lambda x: x[1], reverse=True)
    top_documents = []
    heap = []
    for doc_id, tfidf_score in term_info:
        similarity_score = cossim_data.at[masters.index[doc_id], masters.index[doc_id]]
        heapq.heappush(heap, (similarity_score, doc_id))
        if len(heap) > 5:
            heapq.heappop(heap)
    top_documents = sorted(heap, key=lambda x: x[0], reverse=True)
    
    # Display the results as a table
    result_table = pd.DataFrame(columns=['courseName', 'universityName', 'description', 'url', 'Similarity'])
    
    for similarity, doc_id in top_documents:
        row_data = masters.loc[masters.index[doc_id], ['courseName', 'universityName', 'description', 'url']].tolist()
        row_data.append(similarity)
        result_table = pd.concat([result_table, pd.DataFrame([row_data], columns=result_table.columns)], ignore_index=True)
    return result_table

In [36]:
inverted_index = create_inverted_index(tfidf_data)
word_to_search = "account"
related_documents = find_related_documents(word_to_search, masters, tfidf_data, cossim_data, inverted_index)
related_documents

C:\Users\Elias Antoun\AppData\Local\Temp\ipykernel_21980\4163418527.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_table = pd.concat([result_table, pd.DataFrame([row_data], columns=result_table.columns)], ignore_index=True)


,courseName,universityName,description,url,Similarity
0,Physics MSc,University of Groningen,"[univers, groningen, long-stand, tradit, high-...",,1.0


**PART 3: ALternative Scoring function**

In this part we will be implementing an alternative, and more rigorous scoring function compared to the one presented in 2.2.

This function will be scoring the reuslts accordidng to the following criteria:

1- Description matching as before
2- Exact matches in course titles
3- Possible matches in geographical location
4- Possible tuition fee matching, preferring universities with tuition costs lower than any that may be specified in the query.
5- Possible matches in modality and administration
6- Preference to higher ranked universities (Qs 2024 ranking)

*Score 1: Description matching*

In [21]:
def description_score(query):
    query_terms = set(stemmer.stem(query))
    description_terms = masters["description"]
    match_count = len(query_terms.intersection(description_terms))
    return match_count

*Score 2: Exact Matches in Course Titles*

In [20]:
def title_exact_match_score(query):
    return 1 if query.lower() == masters["courseName"].lower() else 0

*Score 3: Geographical location*

In [19]:
def location_score(user_location):
    return 1 if user_location.lower() in masters["country"].lower() or masters["city"] else 0

*Score 4: Tuition fees matching*

Case 1: masters["fees"] is not empty

In [22]:
def fee_score(user_budget):
    if masters["fees"] >= 0:
        return 1 if user_budget >= masters["fees"] else 0

Case 2: masters["fees"] is not reported

Case 3: Administration score

In [ ]:
def administration_score(query)

In [ ]:
Adm = {"full time": [""]}